In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power, GofChisquarePower
from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import norm, chi2_contingency, mannwhitneyu
from random import randint
from scipy import stats 

In [2]:
N = 200
mu = 50 
mean = 10
lift = 1.1
pwr = 0.8 
alpha = 0.05
sd = 10

## T-test

In [3]:
data_group1 = [abs(i) for i in np.random.normal(loc=mean, scale=sd, size=N)]
data_group2 = [abs(i) for i in np.random.normal(loc=mean*lift, scale=sd, size=N)]

In [19]:
(data_group1[:5], data_group2[:5])

([20.039736250478086,
  9.775376127546025,
  16.208953358235913,
  17.876362371111995,
  21.745483943006462],
 [2.1804092344967376,
  5.939386766301629,
  10.438489865200854,
  5.147138795610138,
  4.5320947135214205])

In [4]:
p_val = ttest_ind(data_group1, data_group2)[1]
p_val

0.3246837246991986

In [5]:
effect_size = mean / sd * lift

nobs1 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=pwr, nobs1=None, ratio=1)

nobs1 #number of observations per split

14.001886583878685

## Mann Whitney U-test

In [6]:
U1, p = mannwhitneyu(data_group1, data_group2)
print(U1, p)

19050.0 0.4114952366298432


## Chi-Square

In [7]:
a = [int(0.5 >= np.random.rand()) for i in range(N)]
b = [int(0.6 >= np.random.rand()) for i in range(N)]

In [8]:
a[:10], b[:10]

([0, 0, 0, 1, 0, 0, 1, 1, 1, 0], [0, 1, 0, 0, 0, 1, 1, 0, 1, 1])

In [9]:
successes, fails = [sum(a), sum(b)], [N - sum(a), N - sum(b)]
p_val = chi2_contingency(np.array([fails, successes]))[1]
p_val

0.31555593098319923

In [10]:
probs0, probs1 = np.array([sum(a), N - sum(a)]), np.array([sum(b), N - sum(b)])
effect_size = chisquare_effectsize(probs0, probs1, correction=None, cohen=True, axis=0)
nobs1 = GofChisquarePower().solve_power(effect_size=effect_size, alpha=alpha,
                                        power=pwr, n_bins=2)
nobs1 #number of observations per split

/Users/ghostrider/Library/Python/3.9/lib/python/site-packages/scipy/_lib/_util.py:69: RuntimeWarning: overflow encountered in _ncx2_sf
  np.place(out, cond, f(*temp))


648.0823582535387

## T-test Delta Method for Ratio Metric

In [11]:
# t-test deltha method

# mde

In [12]:
import pandas as pd
import numpy as np
from random import randint
from scipy import stats 

#dummy variables
click_control = [randint(0,20) for i in range(10000)]
view_control = [randint(1,60) for i in range(10000)]

click_treatment = [randint(0,21) for i in range(10000)]
view_treatment = [randint(1,60) for i in range(10000)]

control = pd.DataFrame({'click':click_control,'view':view_control})
treatment = pd.DataFrame({'click':click_treatment,'view':view_treatment})

#variance estimation of metrics ratio
def var_ratio(x,y): #x/y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    var_x = np.var(x,ddof=1)
    var_y = np.var(y,ddof=1)
    cov_xy = np.cov(x,y,ddof=1)[0][1]
    result = (var_x/mean_x**2 + var_y/mean_y**2 - 2*cov_xy/(mean_x*mean_y))*(mean_x*mean_x)/(mean_y*mean_y*len(x))
    return result
    
#ttest calculation 
def ttest(mean_control,mean_treatment,var_control,var_treatment):
    diff = mean_treatment - mean_control
    var = var_control+var_treatment
    stde = 1.96*np.sqrt(var)
    lower = diff - stde 
    upper = diff + stde
    z = diff/np.sqrt(var)
    p_val = stats.norm.sf(abs(z))*2

    result = {'mean_control':mean_control,
             'mean_experiment':mean_treatment,
             'var_control':var_control,
             'var_experiment':var_treatment,
             'difference':diff,
             'lower_bound':lower,
             'upper_bound':upper,
             'p-value':p_val}
    return pd.DataFrame(result,index=[0])

var_control = var_ratio(control['click'],control['view'])
var_treatment = var_ratio(treatment['click'],treatment['view'])
mean_control = control['click'].sum()/control['view'].sum()
mean_treatment = treatment['click'].sum()/treatment['view'].sum()

ttest(mean_control,mean_treatment,var_control,var_treatment)

,mean_control,mean_experiment,var_control,var_experiment,difference,lower_bound,upper_bound,p-value
0,0.329964,0.347725,0.000008,0.000008,0.017761,0.009992,0.02553,0.000007


In [13]:
# num – числитель, den – знаменатель
num_1 = np.array([1,2,3,4,5])
den_1 = np.array([2,3,4,5,6])
num_2 = np.array([1,2,3,8,5])
den_2 = np.array([2,9,4,5,6])

def deltamethod(x, y, independent = False, bc = False):
    n = len(x)
    mux = np.mean(x)
    muy = np.mean(y)
    
    v11 = np.var(y,ddof=1)
    v22 = np.var(x,ddof=1)
    if independent == True:
        v12 = 0
    else: 
        v12 = np.cov(x,y)[0][1]
    
    rto = muy / mux
    est = rto - 1
    if bc == True:
        est = est +muy/mux**3*v22/n - 1/mux**2*v12/n
    sdest = (v11 / mux**2) - (2 * muy / mux**3 * v12) + (muy**2 / mux**4 * v22)
    sdest = np.sqrt(sdest)
    return est, sdest

# Считаем оценку ratio и дисперсий
rto_1, sdest_1 = deltamethod(num_1,den_1,independent=True,bc=True)
rto_2, sdest_2 = deltamethod(num_2,den_2,independent=True,bc=True)

# Для effect_size
lift = 0.1
effect_size = rto_1 * lift / sdest_1

nobs1 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=pwr, nobs1=None, ratio=1)

nobs1, effect_size

(7298.454871248002, 0.04638007234913623)

## Поправка на множественные сравнения

In [14]:
results = pd.DataFrame({"p_val":[0.00212, 0.04865, 0.10875]}, index=['A_B', 'B_C', 'A_C'])

In [15]:
results['p_val_adjusted'] = multipletests(results['p_val'].values, alpha=alpha, method='fdr_bh')[1]

In [16]:
results

,p_val,p_val_adjusted
A_B,0.00212,0.006360
B_C,0.04865,0.072975
A_C,0.10875,0.108750
